In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lucknow-city-india-450-housing-prices-updated/lucknow_hosuing_prices - Main File.csv


## Importing ML Libraries

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import LinearRegression

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

## Loading dataset

In [4]:
df = pd.read_csv('/kaggle/input/lucknow-city-india-450-housing-prices-updated/lucknow_hosuing_prices - Main File.csv')

In [5]:
df.dtypes

bhk              int64
type            object
area_sq_ft       int64
location        object
price_lakh     float64
carpet_area     object
status          object
bathrooms        int64
dtype: object

In [6]:
df.isna().sum()

bhk              0
type             0
area_sq_ft       0
location         0
price_lakh       0
carpet_area    152
status           0
bathrooms        0
dtype: int64

## Preprocessing Data

In [7]:
df = df.fillna(0)

In [8]:
df.head()

,bhk,type,area_sq_ft,location,price_lakh,carpet_area,status,bathrooms
0,3,Independent,1100,Rajajipuram,33.0,720,StatusReady,2
1,3,Independent,1200,Sarojini Nagar,27.0,0,StatusUnder,2
2,2,Independent,1250,Jankipuram,50.0,1000,StatusReady,2
3,3,Independent,1320,Omaxe City,42.0,900,StatusReady,3
4,3,Independent,900,Vrindavan Yojna,48.0,,StatusReady,2


In [9]:
location_count = df['location'].value_counts()
location_count = location_count.reset_index()
at_least_5 = (len(df)*5)/100
req_location = location_count[location_count['count']>=at_least_5]
req_location

,location,count
0,Faizabad Road,52
1,Jankipuram,49
2,Kursi Road,32
3,Sitapur Road,24


In [10]:
df_req = df[df['location'].isin(req_location['location'])]

In [11]:
df_req.head()

,bhk,type,area_sq_ft,location,price_lakh,carpet_area,status,bathrooms
2,2,Independent,1250,Jankipuram,50.0,1000,StatusReady,2
10,2,Independent,850,Jankipuram,27.0,600,StatusUnder,1
16,2,Independent,1500,Jankipuram,42.0,1250,StatusReady,2
26,2,Independent,1100,Jankipuram,44.0,700,StatusReady,2
29,2,Independent,1100,Jankipuram,46.0,1000,StatusReady,2


In [12]:
df_dummies = pd.get_dummies(df_req[['type','status','location']],drop_first=True,dtype=int)

In [13]:
df_req.head()

,bhk,type,area_sq_ft,location,price_lakh,carpet_area,status,bathrooms
2,2,Independent,1250,Jankipuram,50.0,1000,StatusReady,2
10,2,Independent,850,Jankipuram,27.0,600,StatusUnder,1
16,2,Independent,1500,Jankipuram,42.0,1250,StatusReady,2
26,2,Independent,1100,Jankipuram,44.0,700,StatusReady,2
29,2,Independent,1100,Jankipuram,46.0,1000,StatusReady,2


In [14]:
df_req  = pd.concat([df_req,df_dummies],axis='columns')

In [15]:
df_req['carpet_area'].replace(' ',0,inplace=True)

In [16]:
to_be_scaled = df_req[['area_sq_ft','price_lakh','carpet_area']]

In [17]:
scale = StandardScaler()

In [18]:
scaled_data = scale.fit_transform(to_be_scaled)

In [19]:
df_req[['area_sq_ft','price_lakh','carpet_area']] = scaled_data

In [20]:
df_req.drop(columns=['type','location','status'],inplace=True)

In [21]:
df_req

,bhk,area_sq_ft,price_lakh,carpet_area,bathrooms,type_Villa,status_StatusUnder,location_Jankipuram,location_Kursi Road,location_Sitapur Road
2,2,0.890495,1.216571,0.972847,2,0,0,1,0,0
10,2,-1.171717,-1.152090,0.024038,1,0,1,1,0,0
16,2,2.179377,0.392689,1.565853,2,0,0,1,0,0
26,2,0.117165,0.598659,0.261240,2,0,0,1,0,0
29,2,0.117165,0.804630,0.972847,2,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
468,2,0.632718,0.907615,0.972847,2,1,0,0,0,0
469,2,1.148271,0.907615,1.210049,2,1,0,0,0,0
470,2,0.890495,0.701644,0.972847,2,1,0,0,0,0
471,2,0.632718,-0.225223,-1.399177,2,0,1,0,1,0


In [22]:
X = df_req.drop(columns='price_lakh')
y = df_req['price_lakh']

## Training and Testing Model

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [24]:
model = LinearRegression()

In [25]:
model.fit(X_train,y_train)

LinearRegression()

In [26]:
model.score(X_train,y_train)

0.7112823899084595

In [27]:
model.score(X_test,y_test)

0.824451183624956

In [28]:
len(X_train)

125

In [29]:
bag_model = BaggingRegressor(estimator=LinearRegression(),
                        n_estimators=10,max_samples=1.0,oob_score=False)

In [30]:
bag_model.fit(X_train,y_train)

BaggingRegressor(estimator=LinearRegression())

In [31]:
bag_model.score(X_test,y_test)

0.8238394383647838

In [32]:
cross_val_score(bag_model,X,y,cv=5)

array([0.35532417, 0.72709872, 0.58796814, 0.81747909, 0.51213713])